In [4]:
import io
import base64
import numpy as np
import torch
from PIL import Image

from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler

import os

os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [5]:
def image_to_base64_data_uri(image_input):
    # Check if the input is a file path (string)
    if isinstance(image_input, str):
        with open(image_input, "rb") as img_file:
            base64_data = base64.b64encode(img_file.read()).decode('utf-8')

    # Check if the input is a PIL Image
    elif isinstance(image_input, Image.Image):
        buffer = io.BytesIO()
        image_input.save(buffer, format="PNG")  # You can change the format if needed
        base64_data = base64.b64encode(buffer.getvalue()).decode('utf-8')

    else:
        raise ValueError("Unsupported input type. Input must be a file path or a PIL.Image.Image instance.")

    return f"data:image/png;base64,{base64_data}"

In [6]:
mmproj = "./mmproj-model-f16.gguf"
model_path = "./ggml-model-q4_0.gguf"


chat_handler = Llava15ChatHandler(clip_model_path=mmproj, verbose=True)
spacellava = Llama(
    model_path=model_path,
    chat_handler=chat_handler,
    n_ctx=2048,
    logits_all=True,
    n_gpu_layers=-1,

)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA RTX A5000, compute capability 8.6, VMM: yes


clip_model_load: loaded meta data with 19 key-value pairs and 377 tensors from ./mmproj-model-f16.gguf
clip_model_load: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
clip_model_load: - kv   0:                       general.architecture str              = clip
clip_model_load: - kv   1:                      clip.has_text_encoder bool             = false
clip_model_load: - kv   2:                    clip.has_vision_encoder bool             = true
clip_model_load: - kv   3:                   clip.has_llava_projector bool             = true
clip_model_load: - kv   4:                          general.file_type u32              = 1
clip_model_load: - kv   5:                               general.name str              = openai/clip-vit-large-patch14-336
clip_model_load: - kv   6:                        general.description str              = image encoder for LLaVA
clip_model_load: - kv   7:                        clip.projector_type str              = mlp
clip_

key clip.vision.image_grid_pinpoints not found in file
key clip.vision.mm_patch_merge_type not found in file
key clip.vision.image_crop_resolution not found in file
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:


clip_model_load: compute allocated memory: 32.89 MB


  Device 0: NVIDIA RTX A5000, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 22 key-value pairs and 363 tensors from ./ggml-model-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv 

In [7]:
# @title Set your image and prompt
from typing import Iterator


from llama_cpp import CreateChatCompletionResponse
from llama_cpp.llama_types import CreateChatCompletionStreamResponse


image_path = './warehouse_test.jpg' # @param {type:"string"}
prompt = 'How high is the stack of boxes on the pallet held up by the forklift in meters?' # @param {type:"string"}

data_uri = image_to_base64_data_uri(image_path)
messages = [
    {"role": "system", "content": "You are an assistant who perfectly describes images."},
     {
         "role": "user",
         "content": [
             {"type": "image_url", "image_url": {"url": data_uri}},
              {"type" : "text", "text": prompt}
             ]
         }
    ]
results: CreateChatCompletionResponse | Iterator[CreateChatCompletionStreamResponse] = spacellava.create_chat_completion(messages = messages)


encode_image_with_clip: image embedding created: 576 tokens

encode_image_with_clip: image encoded in   765.07 ms by CLIP (    1.33 ms per image patch)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3378.54 ms
llama_print_timings:      sample time =       5.05 ms /    23 runs   (    0.22 ms per token,  4551.75 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     381.26 ms /    23 runs   (   16.58 ms per token,    60.33 tokens per second)
llama_print_timings:       total time =     442.20 ms /    24 tokens


In [9]:
type(results)

dict

In [1]:

print(results["choices"][0]["message"]["content"])

NameError: name 'results' is not defined

In [2]:

import torch

dev = torch.device("cuda:4")

In [3]:
str(dev)

'cuda:4'